<a href="https://colab.research.google.com/github/Redcoder815/Deep_Learning_TensorFlow/blob/main/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np

class MyRNN(tf.keras.Model):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.hidden_size = hidden_size

        # Parameters
        self.Wxh = self.add_weight(
            shape=(input_size, hidden_size),
            initializer=tf.random_normal_initializer(stddev=0.1),
            trainable=True
        )
        self.Whh = self.add_weight(
            shape=(hidden_size, hidden_size),
            initializer=tf.random_normal_initializer(stddev=0.1),
            trainable=True
        )
        self.bh = self.add_weight(
            shape=(hidden_size,),
            initializer=tf.zeros_initializer(),
            trainable=True
        )

        self.Why = self.add_weight(
            shape=(hidden_size, output_size),
            initializer=tf.random_normal_initializer(stddev=0.1),
            trainable=True
        )
        self.by = self.add_weight(
            shape=(output_size,),
            initializer=tf.zeros_initializer(),
            trainable=True
        )

    def call(self, x, h0=None):
        """
        x: (seq_len, batch, input_size)
        """
        seq_len = tf.shape(x)[0]
        batch = tf.shape(x)[1]

        if h0 is None:
            h = tf.zeros((batch, self.hidden_size))
        else:
            h = h0

        outputs = []

        for t in range(seq_len):
            xt = x[t]  # (batch, input_size)

            h = tf.tanh(
                tf.matmul(xt, self.Wxh) +
                tf.matmul(h, self.Whh) +
                self.bh
            )

            y = tf.matmul(h, self.Why) + self.by
            outputs.append(y)

        return tf.stack(outputs), h


# -----------------------------
# Data prep
# -----------------------------
text = "hello world"
chars = sorted(list(set(text)))
stoi = {c:i for i,c in enumerate(chars)}
itos = {i:c for c,i in stoi.items()}

def encode(s):
    return np.array([stoi[c] for c in s], dtype=np.int32)

def one_hot(idx, vocab_size):
    return tf.one_hot(idx, vocab_size)

vocab_size = len(chars)
hidden_size = 32

model = MyRNN(vocab_size, hidden_size, vocab_size)
optimizer = tf.keras.optimizers.Adam(0.01)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

data = encode(text)

# -----------------------------
# Training loop
# -----------------------------
for epoch in range(500):
    with tf.GradientTape() as tape:
        x = one_hot(data[:-1], vocab_size)  # (seq, vocab)
        x = tf.expand_dims(x, 1)            # (seq, batch=1, vocab)
        y_true = data[1:]                   # next char

        y_pred, _ = model(x)
        y_pred = tf.squeeze(y_pred, axis=1)  # (seq, vocab)

        loss = loss_fn(y_true, y_pred)

    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    if epoch % 50 == 0:
        print(f"epoch {epoch}, loss={loss.numpy():.4f}")


# -----------------------------
# Generation
# -----------------------------
def generate(model, start="h", length=20):
    idx = stoi[start]
    h = None
    out = start

    for _ in range(length):
        x = one_hot([idx], vocab_size)
        x = tf.expand_dims(x, 1)  # (seq=1, batch=1, vocab)

        y, h = model(x, h)
        logits = y[-1]  # (1, vocab)

        probs = tf.nn.softmax(logits)
        idx = tf.random.categorical(tf.math.log(probs), 1).numpy()[0,0]

        out += itos[idx]

    return out

print(generate(model, "h"))

epoch 0, loss=2.0967
epoch 50, loss=0.0067
epoch 100, loss=0.0025
epoch 150, loss=0.0016
epoch 200, loss=0.0011
epoch 250, loss=0.0008
epoch 300, loss=0.0006
epoch 350, loss=0.0005
epoch 400, loss=0.0004
epoch 450, loss=0.0003
hello worldrlworlorld


In [2]:
import tensorflow as tf
import numpy as np

text = "hello world. this is a simple test for an rnn built from scratch."
chars = sorted(list(set(text)))
vocab_size = len(chars)

char2idx = {c:i for i,c in enumerate(chars)}
idx2char = np.array(chars)

encoded = np.array([char2idx[c] for c in text])

seq_length = 40
step = 3

inputs = []
targets = []

for i in range(0, len(encoded) - seq_length, step):
    inputs.append(encoded[i:i+seq_length])
    targets.append(encoded[i+seq_length])

inputs = np.array(inputs)
targets = np.array(targets)

embedding_dim = 32
rnn_units = 64

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=seq_length),
    tf.keras.layers.SimpleRNN(rnn_units),
    tf.keras.layers.Dense(vocab_size, activation="softmax")
])

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=tf.keras.optimizers.Adam(0.01)
)

model.summary()

history = model.fit(inputs, targets, epochs=30, batch_size=32)

def generate_text(model, start_string, length=200, temperature=1.0):
    input_eval = np.array([char2idx[c] for c in start_string])
    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = []

    for _ in range(length):
        preds = model(input_eval)
        preds = preds / temperature # Removed the extra slicing
        predicted_id = tf.random.categorical(preds, num_samples=1)[0,0].numpy()

        text_generated.append(idx2char[predicted_id])
        input_eval = tf.concat([input_eval, [[predicted_id]]], axis=1)
        input_eval = input_eval[:, -seq_length:]  # keep window size fixed

    return start_string + "".join(text_generated)

print(generate_text(model, "hello "))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_1 (SimpleRNN)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - loss: 2.9766
Epoch 2/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - loss: 2.5642
Epoch 3/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step - loss: 2.2436
Epoch 4/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step - loss: 1.7112
Epoch 5/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step - loss: 1.6078
Epoch 6/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step - loss: 1.2882
Epoch 7/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step - loss: 1.0936
Epoch 8/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step - loss: 0.7423
Epoch 9/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step - loss: 0.6418
Epoch 10/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step - loss: 0.4340
Epoch 11/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - loss: 0.2721
Epoch 12/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - loss: 0.1814
Epoch 13/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step - loss: 0.1262
Epoch 14/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - loss: 0.0961
Epoch 15/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - loss: 0.0608
Epoch 16/30
1/1 ━━━━━━━━━━━━━━━